In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = pd.read_json('predictions_basic.json')
result.head()

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8293683, 1330944, 9676948, 867081, 966472]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13115504, 1040819, 6443095, 12673030, 1018691]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1047202, 905593, 15663041, 9707008, 975603]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9883969, 10312199, 5586784, 877662, 74671]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15716087, 1595969, 861227, 16099862, 9655603]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]"


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
weights = data.groupby('item_id')['sales_value'].sum().reset_index()
weights['sales_value'] = weights['sales_value'].apply(np.log)
weights.rename(columns={'sales_value': 'log_sales'}, inplace=True)
weights.head()

,item_id,log_sales
0,25671,3.041661
1,26081,-0.010050
2,26093,0.463734
3,26190,0.431782
4,26355,0.683097


In [6]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(items_weights.item_id, size=n, replace=False)
    recs = items_weights.loc[items_weights.item_id.isin(recs)].sort_values(by='log_sales', ascending=False).item_id
    
    return recs.tolist()

In [7]:
weighted_random_recommendation(weights)

[16122429, 17285551, 987494, 828837, 45971]

In [8]:
%%time

result['WRC'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weights))
result.head()

Wall time: 7.76 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,WRC
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8293683, 1330944, 9676948, 867081, 966472]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[869024, 943241, 13133613, 12782089, 1122711]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13115504, 1040819, 6443095, 12673030, 1018691]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[886159, 1071522, 989365, 1019763, 1118573]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1047202, 905593, 15663041, 9707008, 975603]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[1082185, 1029743, 6534178, 1127831, 995785]","[969980, 899324, 1108781, 17209436, 797263]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[9883969, 10312199, 5586784, 877662, 74671]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[1082185, 1029743, 1127831, 995785, 1044078]","[978294, 1133893, 971152, 6517637, 12171785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15716087, 1595969, 861227, 16099862, 9655603]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 1029743, 1098066, 6534178, 1127831]","[835028, 9396694, 12810401, 852420, 2078710]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
from metrics import precision_at_k

In [10]:
for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0008:random_recommendation
0.1552:popular_recommendation
0.1369:itemitem
0.1329:cosine
0.139:tfidf
0.2019:own_purchases
0.0011:WRC


C:\Users\mahhets\Desktop\учеба\Рекомендательные системы\2.Детерминированные алгоритмы item-item\metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


Как видим, наличие семплирования по весу не дало значительного прироста случайной модели

В некоторых итерациях результат был даже хуже